[![mlpack-lab Image](https://img.shields.io/endpoint?url=https%3A%2F%2Flab.kurg.org%2Fstatus%2Fstatus.json)](https://lab.mlpack.org)

You can easily run this notebook at https://lab.mlpack.org/

This notebook shows how to get started with training reinforcement learning agents, particularly DQN agents, using mlpack. Here, we train a [Simple DQN](https://www.cs.toronto.edu/~vmnih/docs/dqn.pdf) agent to get high scores for the [CartPole](https://gym.openai.com/envs/CartPole-v0) environment. 

mlpack contains non-GUI implementations of some of OpenAI gym's environments. In this notebook, we use one such environment for training the agent, as it is fast to train on.

As for testing, we make the agent run on OpenAI Gym toolkit's GUI interface provided through a distributed infrastructure (TCP API). More details can be found [here](https://github.com/zoq/gym_tcp_api).

A video of the trained agent can be seen in the end.

## Including necessary libraries and namespaces

In [1]:
#include <mlpack/core.hpp>

In [ ]:
#include <mlpack/methods/ann/ffn.hpp>
#include <mlpack/methods/reinforcement_learning/q_learning.hpp>
#include <mlpack/methods/reinforcement_learning/q_networks/simple_dqn.hpp>
#include <mlpack/methods/reinforcement_learning/environment/cart_pole.hpp>
#include <mlpack/methods/reinforcement_learning/policy/greedy_policy.hpp>
#include <mlpack/methods/reinforcement_learning/training_config.hpp>

In [ ]:
// Used to run the agent on gym's environment (provided externally) for testing.
#include <gym/environment.hpp>

In [ ]:
// Used to generate and display a video of the trained agent.
#include "xwidgets/ximage.hpp"
#include "xwidgets/xvideo.hpp"
#include "xwidgets/xaudio.hpp"

In [ ]:
using namespace mlpack;

In [ ]:
using namespace mlpack::ann;

In [ ]:
using namespace ens;

In [ ]:
using namespace mlpack::rl;

## Initializing the agent

In [ ]:
// Set up the network.
SimpleDQN<> model(4, 128, 32, 2);

In [ ]:
// Set up the policy and replay method.
GreedyPolicy<CartPole> policy(1.0, 1000, 0.1, 0.99);
RandomReplay<CartPole> replayMethod(32, 1000);

In [ ]:
// Set up training configurations.
TrainingConfig config;
config.StepSize() = 0.001;
config.Discount() = 0.99;
config.TargetNetworkSyncInterval() = 100;
config.ExplorationSteps() = 100;
config.DoubleQLearning() = false;
config.StepLimit() = 200;

In [ ]:
// Set up DQN agent.
QLearning<CartPole, decltype(model), AdamUpdate, decltype(policy)>
  agent(std::move(config), std::move(model), std::move(policy),
  std::move(replayMethod));

## Preparation for training the agent

In [ ]:
// Initializing training variables.
arma::running_stat<double> averageReturn;
size_t episodes = 0;

In [ ]:
// Function to train the agent on mlpack's own implementation of the CartPole environment.
void train(const size_t threshold)
{
  agent.Deterministic() = false;
  while (true)
  {
    double episodeReturn = agent.Episode();
    averageReturn(episodeReturn);
    episodes += 1;

    if(episodes % 10 == 0)
    {
      std::cout << "Average return: " << averageReturn.mean()
          << "\t Episode return: " << episodeReturn 
          << "\t Episode number: " << episodes << std::endl;
    }

    if (episodes > 1000)
    {
      std::cout << "Cart Pole with DQN failed." << std::endl;
      break;
    }

    if (averageReturn.mean() > threshold)
    {
      agent.Deterministic() = true;
      arma::running_stat<double> testReturn;
      for (size_t i = 0; i < 100; ++i)
        testReturn(agent.Episode());

      std::cout << "Average return in 100 consecutive trials : "
          << testReturn.mean() << std::endl;
      break;
    }
  }
}

## Let the training begin

In [ ]:
// Training the agent till the total average return reaches 35.
train(35)

## Testing the trained agent

In [ ]:
// Creating and setting up the gym environment for testing.
gym::Environment env("gym.kurg.org", "4040", "CartPole-v0");
env.monitor.start("./dummy/", true, true);

// Resets the environment.
env.reset();
env.render();

double totalReward = 0;
size_t totalSteps = 0;

// Testing the agent on gym's environment.
while (1)
{
  // State from the environment is passed to the agent's internal representation.
  agent.State().Data() = env.observation;

  // With the given state, the agent performs an action according to its defined policy.
  agent.Step();

  // Action to take, decided by the policy.
  arma::mat action = {double(agent.Action())};

  env.step(action);
  totalReward += env.reward;
  totalSteps += 1;

  if (env.done)
  {
    std::cout << " Total steps: " << totalSteps << "\t Total reward: "
        << totalReward << std::endl;
    break;
  }

  // Uncomment the following lines to see the reward and action in each step.
  // std::cout << " Current step: " << totalSteps << "\t current reward: "
  //   << totalReward << "\t Action taken: " << action;
}

env.close();
std::string url = env.url();

auto video = xw::video_from_url(url).finalize();
video

## A little more training...

In [ ]:
// Training the same agent till the total average return reaches 150
train(140)

# Final agent testing!

In [ ]:
// Creating and setting up the gym environment for testing.
gym::Environment env("gym.kurg.org", "4040", "CartPole-v0");
env.monitor.start("./dummy/", true, true);

// Resets the environment.
env.reset();
env.render();

double totalReward = 0;
size_t totalSteps = 0;

// Testing the agent on gym's environment.
while (1)
{
  // State from the environment is passed to the agent's internal representation.
  agent.State().Data() = env.observation;

  // With the given state, the agent performs an action according to its defined policy.
  agent.Step();

  // Action to take, decided by the policy.
  arma::mat action = {double(agent.Action())};

  env.step(action);
  totalReward += env.reward;
  totalSteps += 1;

  if (env.done)
  {
    std::cout << " Total steps: " << totalSteps << "\t Total reward: "
        << totalReward << std::endl;
    break;
  }

  // Uncomment the following lines to see the reward and action in each step.
  // std::cout << " Current step: " << totalSteps << "\t current reward: "
  //   << totalReward << "\t Action taken: " << action;
}

env.close();
std::string url = env.url();

auto video = xw::video_from_url(url).finalize();
video